<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/eras/eras/tune/programs/0111_convLayers_auto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 前処理
%tensorflow_version 2.x

## import
# file dealing
import os
from os import path
import shutil
from google.colab import drive
import datetime
# data dealing
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
# process deasing
import gc
from time import sleep

# machine learning (back)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, initializers, callbacks

# machine learning
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical

import json
from collections import OrderedDict
import pprint
import re
import requests

## ツール
def beep():
  from google.colab import output
  output.eval_js('new Audio(\
"https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg")\
.play()') 

def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'cHdELzsau6ve8hNVL3FxPz65Jdyquzuj2kd021u8q1L'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': notification_message}
    requests.post(line_notify_api, headers = headers, data = data)


  
## フォーマット規定
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値





format1 = video_format()
## gdrive 接続
if not path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print("Already confirm")

## colab テンポラリディレクトリの作成
desk = '/content/desk'
if not os.path.exists(desk):
  os.mkdir(desk)
os.chdir(desk)
print("Created at /content/desk")

Already confirm
Created at /content/desk


#実験内容

In [2]:
# learningDict = {
#     "optimizer" : {
#         "this.optimizer" : "adam",
#         "learning_rate" : middle,
#         "beta_1" : 0.9,
#         "beta_2" : 0.999
#     },
#     "theWay" : {
#         "batch_size" : 32,
#         "epochs" : 1024,
#     },
#     "compared_losses" : [
#                          tf.keras.losses.CategoricalCrossentropy(),
#                          tf.keras.losses.MeanSquaredError(), 
#     ]
# }
learningDict = {
    "optimizer" : {
        "this.optimizer" : "sgd",
        "learning_rate" : 1e-5,
        "momentum" : 0,
        "decay" : 0.1
        "nesterov" : False
    },
    "theWay" : {
        "batch_size" : 4,
        "epochs" : 1024,
    },
    "compared_losses" : [
                        #  tf.keras.losses.CategoricalCrossentropy(),
                        #  tf.keras.losses.MeanSquaredError(), 
                         tf.keras.losses.MeanAbsoluteError(),
                        #  tf.keras.losses.SquaredHinge()
                         
    ]
}

# 最適化処理 (adamのみ対応)
if learningDict["optimizer"]["this.optimizer"] == "adam":
  optimizer = keras.optimizers.Adam(
      lr=learningDict["optimizer"]["learning_rate"],
      beta_1=learningDict["optimizer"]["beta_1"],
      beta_2=learningDict["optimizer"]["beta_2"])
  print("adam is used as a optimizer")

elif learningDict["optimizer"]["this.optimizer"] == "Nadam":
  optimizer = keras.optimizers.Nadam(
      lr=learningDict["optimizer"]["learning_rate"],
      beta_1=learningDict["optimizer"]["beta_1"],
      beta_2=learningDict["optimizer"]["beta_2"],
      epsilon=None, 
      schedule_decay=0.4)
  print("Nadam is used as a optimizer")

elif learningDict["optimizer"]["this.optimizer"] == "sgd":
  optimizer = keras.optimizers.SGD(
      lr=learningDict["optimizer"]["learning_rate"],
      momentum=learningDict["optimizer"]["momentum"],
      decay=learningDict["optimizer"]["decay"],
      nesterov=learningDict["optimizer"]["nesterov"]) 
  
  print("sgd is used as a optimizer")


else:
  print("error")

sgd is used as a optimizer


#データをインポート

In [3]:
# 必要なファイルを'desk'にコピー
%%time
wants_paths = [
'/content/drive/MyDrive/colab/dence1223/normalTrainLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTrainVideos.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestLabels.npz',
'/content/drive/MyDrive/colab/dence1223/normalTestVideos.npz'
]

for want in wants_paths:
  if not os.path.exists(os.path.join(desk, os.path.basename(want))):
    shutil.copy2(want, desk)
    print("get : ", want)
            

CPU times: user 189 µs, sys: 42 µs, total: 231 µs
Wall time: 188 µs


In [4]:
# ファイルをメモリにコピー
def prepare_data():
  # traindata
  v = np.load('/content/desk/normalTrainVideos.npz')
  l = np.load('/content/desk/normalTrainLabels.npz')

  train_videos = [] 
  train_labels = []
  for i in v.files:
    train_videos.append(v[i])
  for i in l.files:
    train_labels.append(l[i])

  train_videos = np.array(train_videos)
  train_labels = np.array(train_labels)
  train_labels = tf.keras.utils.to_categorical(train_labels, 4)

  # testdata
  v = np.load('/content/desk/normalTestVideos.npz')
  l = np.load('/content/desk/normalTestLabels.npz')

  test_videos = []
  test_labels = []
  for i in v.files:
    test_videos.append(v[i])
  for i in l.files:
    test_labels.append(l[i])

  test_videos = np.array(test_videos)
  test_labels = np.array(test_labels)
  test_labels = tf.keras.utils.to_categorical(test_labels, 4)
  
  return train_videos, train_labels,  test_videos, test_labels

if __name__ == '__main__':
  train_videos, train_labels,  test_videos, test_labels = prepare_data()


#学習実行

実験

In [5]:
seed = 20201218
np.random.seed(seed)
tf.random.set_seed(seed)

In [6]:
%%time
histories = []

# 繰り返しの条件
for i, each_loss in enumerate(learningDict["compared_losses"]):
  send_line_notify('学習開始 ' + str(i+1) + "/" + str(len(learningDict["compared_losses"])))
  # モデル作成
  model = models.Sequential()
  # 入力層
  model.add(
      layers.Reshape(
          (format1.FRAME_SIZE,
          format1.HEIGHT,
          format1.WIDTH,
          format1.COLORinfo),
          input_shape=(format1.FRAME_SIZE * format1.HEIGHT * format1.WIDTH * format1.COLORinfo,),
          name='Input_Layer' )
  )

  # 畳み込み0
  # model.add(
  #     layers.Conv3D(
  #         filters=128,
  #         kernel_size=(1, 9, 16),
  #         strides=(1, 9, 16),
  #         padding='same',
  #         activation='relu',
  #         name='conv0'))
  # # pool0
  # model.add(
  #     layers.MaxPooling3D(pool_size=(1, 3, 3), name='pool0'))


  ## 全結合0
  model.add(
      layers.Flatten(name='pipe'),
  )
  model.add(
      layers.Dense(128,
        activation='relu',
        name='dence0' ),
  )
  # model.add(
  #     layers.Dense(128,
  #       activation='relu',
  #       name='dence1' ),
  # )
  # model.add(
  #     layers.Dense(128,
  #       activation='relu',
  #       name='dence2' ),
  # )
  # 出力層
  model.add(
      layers.Dense(4, activation='softmax', name='Output_Leayer')
  )

  model.compile(
      optimizer=optimizer,
      loss=each_loss,
      metrics=['acc'])

  # 実行
  history = model.fit(
        train_videos, train_labels,
        validation_data=(test_videos, test_labels),
        batch_size=learningDict["theWay"]["batch_size"],
        epochs=learningDict["theWay"]["epochs"],
        verbose=0
        )
  histories.append(history)
  print("Complete.")
  

  # 消去
  sleep(10)
  del model
  keras.backend.clear_session()
  gc.collect()
  send_line_notify('学習終了 ' + str(i+1) + "/" + str(len(learningDict["compared_losses"])))


# beep()


# 状況を保存
# for num, each in enumerate(learningDict["compared_losses"]):
  






# # モデル画像保存
# plot_model(model, 
#            show_shapes=True,
#            show_layer_names=False,
#            to_file='model.png')
# shutil.copy2("model.png", shelf_book_page)

Complete.
CPU times: user 14min 30s, sys: 7min, total: 21min 31s
Wall time: 21min 25s


In [7]:
import re
def save_histories(histories):
  dt_now_jst = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  now = dt_now_jst.strftime('%m%d-%H%M')
  csvs = []

  # もし実験回数が実験予定回数と同じなら
  if len(histories) == len(learningDict["compared_losses"]):
    for i, each_history in enumerate(histories):
      hist_df = pd.DataFrame(each_history.history)
      used_loss = learningDict["compared_losses"][i]
      used_loss = str(used_loss)
      used_loss = re.sub("<tensorflow.python.keras.losses.", "", used_loss)
      used_loss = re.sub("object at .*", "", used_loss)
      used_loss = re.sub("\s", "", used_loss)

      name = str(used_loss) + "_" + now + ".csv"
      csvs.append(name)
      print(name)
      
      hist_df.to_csv(name)
      learningDict["compared_losses"][i] = used_loss
      
 
    #/for i,
    shelf = '/content/drive/MyDrive/colab'
    book = 'histories'
    shelf_book = os.path.join(shelf, book)
    shelf_book_page = os.path.join(shelf_book, now)
    print(shelf_book_page)

    
    # 保存
    if not os.path.exists(shelf_book_page):
      os.mkdir(shelf_book_page)
    for one in csvs:
      shutil.copy2(one, shelf_book_page)
    
    

    with open("situation.json", 'w') as f:
      json.dump(learningDict, f, ensure_ascii=False, indent=4)

    shutil.copy2("situation.json", shelf_book_page)
    send_line_notify('保存完了：' + now)

  #/if len(his
  else:
    send_line_notify('エラー：' + now)

save_histories(histories)

MeanAbsoluteError_0116-1659.csv
/content/drive/MyDrive/colab/histories/0116-1659


おわり